In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt


/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
df1=pd.read_csv('/content/movies.csv')
df2=pd.read_csv('/content/Ratings .csv')
df3=pd.read_csv('/content/user_data.csv')

In [ ]:
print('shape of ratings dataset ::',df2.shape)
print('shape of movies dataset ::',df1.shape)
print('shape of userdata dataset ::',df3.shape)

shape of ratings dataset :: (1000209, 4)
shape of movies dataset :: (3883, 3)
shape of userdata dataset :: (6040, 5)


In [ ]:
#merge datasets:: ratings,movies and userdata
d=pd.merge(df3,df2,on='UserID',how='left')
df=pd.merge(d,df1,on='MovieID',how='left')

In [ ]:
split_data,split_data.Genres = df.iloc[[i   for i,j in enumerate(df['Genres']) for value in j.split("|")] , :],[value for i in df['Genres'] for value in i.split("|")]
split_data.head(10)

/usr/local/lib/python3.6/dist-packages/pandas/core/generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


,UserID,Gender,Age,Occupation,Zip-code,MovieID,Rating,Timestamp,Title,Genres
0,1,F,1,10,48067,1193,5,978300760,One Flew Over the Cuckoo's Nest (1975),Drama
1,1,F,1,10,48067,661,3,978302109,James and the Giant Peach (1996),Animation
1,1,F,1,10,48067,661,3,978302109,James and the Giant Peach (1996),Children's
1,1,F,1,10,48067,661,3,978302109,James and the Giant Peach (1996),Musical
2,1,F,1,10,48067,914,3,978301968,My Fair Lady (1964),Musical
2,1,F,1,10,48067,914,3,978301968,My Fair Lady (1964),Romance
3,1,F,1,10,48067,3408,4,978300275,Erin Brockovich (2000),Drama
4,1,F,1,10,48067,2355,5,978824291,"Bug's Life, A (1998)",Animation
4,1,F,1,10,48067,2355,5,978824291,"Bug's Life, A (1998)",Children's
4,1,F,1,10,48067,2355,5,978824291,"Bug's Life, A (1998)",Comedy


Item based approach : KNN


In [ ]:
!pip install pyreclab

     |████████████████████████████████| 194kB 2.8MB/s 


In [ ]:
df2.columns

Index(['UserID', 'MovieID', 'Rating', 'Timestamp'], dtype='object')

In [ ]:
aa=df.loc[df['UserID']==1]
aa.loc[aa['Rating'] == 4]
aa.head(15)

In [ ]:
!pip install surprise

     |████████████████████████████████| 6.5MB 2.7MB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.0-cp36-cp36m-linux_x86_64.whl size=1675746 sha256=27072f2d1e909f7d23f1fa4a2cd34c678100954497f07142d9ce3f2643ab4c5c
  Stored in directory: /root/.cache/pip/wheels/cc/fa/8c/16c93fccce688ae1bde7d979ff102f7bee980d9cfeb8641bcf
Successfully built scikit-surprise


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from surprise import Reader, Dataset, SVD, model_selection
d = pd.read_csv('/content/Ratings .csv')
print(d.head)
reader = Reader()
data = Dataset.load_from_df(d[['UserID', 'MovieID', 'Rating']], reader)
#data.split(n_folds=5)

<bound method NDFrame.head of          UserID  MovieID  Rating  Timestamp
0             1     1193       5  978300760
1             1      661       3  978302109
2             1      914       3  978301968
3             1     3408       4  978300275
4             1     2355       5  978824291
...         ...      ...     ...        ...
1000204    6040     1091       1  956716541
1000205    6040     1094       5  956704887
1000206    6040      562       5  956704746
1000207    6040     1096       4  956715648
1000208    6040     1097       4  956715569

[1000209 rows x 4 columns]>


In [ ]:
svd = SVD()
print("CROSS VALIDATING THE DATASET", '\n')
model_selection.cross_validate(svd, data, measures=['rmse'], verbose=True)
print("DONE CROSS VALIDATING THE DATASET",'\n')

print("TRAINING THE MODEL")
trainset = data.build_full_trainset()
svd.fit(trainset)
print("DONE TRAINING THE MODEL", '\n')

CROSS VALIDATING THE DATASET 

Evaluating RMSE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8721  0.8737  0.8752  0.8743  0.8750  0.8741  0.0011  
Fit time          48.88   49.35   49.49   49.34   49.26   49.26   0.21    
Test time         2.34    2.52    2.08    2.31    2.35    2.32    0.14    
DONE CROSS VALIDATING THE DATASET 

TRAINING THE MODEL
DONE TRAINING THE MODEL 



In [ ]:
# r_ui -> The true rating
# iid  -> The item id (movie id)
# uid  -> The user id
print("Predicting rating for ",svd.predict(uid=3, iid=1000, r_ui=1))

Predicting rating for  user: 3          item: 1000       r_ui = 1.00   est = 3.04   {'was_impossible': False}


In [ ]:
from collections import defaultdict

def get_top_n(predictions, n=10):
    # First map the predictions to each user.
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))
    # Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]
    return top_n

In [ ]:
testset = trainset.build_anti_testset()

In [ ]:
predictions = svd.test(testset[20:50])

In [ ]:
predictions[1:2]

[Prediction(uid=1, iid=3107, r_ui=3.581564453029317, est=3.583991408336505, details={'was_impossible': False})]

In [ ]:
top_n = get_top_n(predictions, n=5)
top_n

defaultdict(list,
            {1: [(3147, 4.785588204194742),
              (110, 4.776429827436706),
              (1945, 4.385991282333312),
              (1293, 4.349425456345304),
              (1610, 4.317674417461879)]})

In [ ]:
top_n = get_top_n(predictions, n=5)
# Print the recommended items for each user
print("PRINTING RECOMMENDATION")
for uid, user_ratings in top_n.items():
    print(uid, [iid for (iid, _) in user_ratings])
    print(uid)

PRINTING RECOMMENDATION
1 [3147, 110, 1945, 1293, 1610]
1


knn using csr matrix

In [ ]:
pivot_table=df.pivot_table(index='Title',columns='UserID',values='Rating').fillna(0)
pivot_table.head()

UserID,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,...,6001,6002,6003,6004,6005,6006,6007,6008,6009,6010,6011,6012,6013,6014,6015,6016,6017,6018,6019,6020,6021,6022,6023,6024,6025,6026,6027,6028,6029,6030,6031,6032,6033,6034,6035,6036,6037,6038,6039,6040
Title,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
"$1,000,000 Duck (1971)",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
'Night Mother (1986),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0
'Til There Was You (1997),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"'burbs, The (1989)",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...And Justice for All (1979),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
from scipy.sparse import csr_matrix
movie_dfm = csr_matrix(pivot_table.values)
from sklearn.neighbors import NearestNeighbors

model_knn = NearestNeighbors(metric = 'cosine', algorithm = 'brute')
model_knn.fit(movie_dfm)

NearestNeighbors(algorithm='brute', leaf_size=30, metric='cosine',
                 metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                 radius=1.0)

In [38]:
qi = np.random.choice(movie_dfm.shape[0])
print('Similar Movies like :',qi, 'MoviID')

Similar Movies like : 2262 MoviID


In [41]:
distances , indices = model_knn.kneighbors(pivot_table.iloc[qi,:].values.reshape(1, -1), n_neighbors = 5)

In [42]:
for i in range(0, len(distances.flatten())):
    if i == 0:
        print('Recommendations for {0}:\n'.format(pivot_table.index[qi]))
    else:
        print('{0}: {1}, with distance of {2}:'.format(i, pivot_table.index[indices.flatten()[i]], distances.flatten()[i]))

Recommendations for Mummy, The (1999):

1: Rock, The (1996), with distance of 0.4644344897515339:
2: Con Air (1997), with distance of 0.4735393100385815:
3: Blade (1998), with distance of 0.47614086110728504:
4: Independence Day (ID4) (1996), with distance of 0.48895814126404913:
